In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

from sklearn.neural_network import MLPRegressor

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_fit_tuning, fun_load_best_params
from ML_functions import fun_convert_time
from ML_functions import fun_scaled_neg_MAPE, fun_tuning_results, fun_scores

# Assign string "TSP" or "CVRP" to the following variable to define the optimization problem
optimization_problem = "BPP"

# Load data
data, start_script = fun_load_data(optimization_problem)

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data, train_size=0.8)

# **Analyse the data set**

In [11]:
# View names of the features and the target variable
print(f"Number of columns: {len(train_data.columns)}")
display(train_data.columns)

# Feature categories
instance_features = ["Instance ID", "Number Items", "Item Weight Ratio", "Item Size Ratio", "Bin Weight", "Bin Size"]
weight_and_sum_features = ["Weight Size Sum Ratio", "Item Volume Ratio", "Item Density Ratio"]
utilization_features = ["Item Total Bin Utilization Ratio", "Final Total Bin Utilization"]
combination_features = ["Total Bin Combinations Ratio", "Perfect Total Bin Combinations Ratio"]
statistical_features = ["Weight Sum", "Size Sum", "Weight Std", "Size Std", "Weight Max", 
                        "Size Max", "Weight Min", "Size Min", "Correlation", "Skewness Weight", "Skewness Size"]
cost_features = ["Marginal Costs/Bins Ratio", "Total Bins"] #"Shapley Value"
 
# All features
all_features = instance_features + weight_and_sum_features + cost_features + statistical_features + utilization_features + combination_features
print(f"Number of features: {len(all_features)}")

Number of columns: 27


Index(['Instance ID', 'Number Items', 'Item Weight Ratio', 'Item Size Ratio',
       'Bin Weight', 'Bin Size', 'Weight Size Sum Ratio', 'Item Volume Ratio',
       'Item Density Ratio', 'Item Total Bin Utilization Ratio',
       'Total Bin Combinations Ratio', 'Perfect Total Bin Combinations Ratio',
       'Weight Sum', 'Size Sum', 'Weight Std', 'Size Std', 'Weight Max',
       'Size Max', 'Weight Min', 'Size Min', 'Correlation', 'Skewness Weight',
       'Skewness Size', 'Final Total Bin Utilization',
       'Marginal Costs/Bins Ratio', 'Total Bins', 'Shapley Value'],
      dtype='object')

Number of features: 26


In [3]:
# Get the min/max of instance size
min_instance_size = min(train_data["Number Items"])
max_instance_size = max(train_data["Number Items"])
number_of_instances_per_size = int((max(train_data["Instance ID"]) + 1) / len(np.unique(train_data["Number Items"])))
print(f"Min/Max instance size: ({min_instance_size}, {max_instance_size})")
print(f"Number of instances per size: {number_of_instances_per_size}")

# Get the min/max of item weight and size
min_item_weight = min(data["Item Weight"])
max_item_weight = max(data["Item Weight"])
min_item_size = min(data["Item Size"])
max_item_size = max(data["Item Size"])
print(f"Min/Max item weight: ({min_item_weight}, {max_item_weight})")
print(f"Min/Max item size: ({min_item_size}, {max_item_size})")

# Get the min/max of bin weight and size
min_bin_weight = min(train_data["Bin Weight"])
max_bin_weight = max(train_data["Bin Weight"])
min_bin_size = min(train_data["Bin Size"])
max_bin_size = max(train_data["Bin Size"])
print(f"Min/Max bin weight: ({min_bin_weight}, {max_bin_weight})")
print(f"Min/Max bin size: ({min_bin_size}, {max_bin_size})")

Min/Max instance size: (7, 15)
Number of instances per size: 3000
Min/Max item weight: (1, 10)
Min/Max item size: (1, 10)
Min/Max bin weight: (17, 25)
Min/Max bin size: (17, 25)


# **Neural Network - Multi Layer Perceptron**
**Hyperparametertuning**

In [2]:
# Define a pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(hidden_layer_sizes=(64, 32, 16), activation="relu", learning_rate="adaptive",
                                  max_iter=1000, random_state=42))

# Define parameter grid
param_grid = {"mlpregressor__solver": ["sgd", "adam"],
              "mlpregressor__alpha": [0.0001, 0.001, 0.01],
              "mlpregressor__batch_size": [32, 64, 128], 
              "mlpregressor__learning_rate_init": [0.0001, 0.001], 
              "mlpregressor__early_stopping": [True, False]}

# Grid search
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, 
                           cv=GroupKFold(n_splits=3).split(X_train, y_train, groups=X_train["Instance ID"]), 
                           scoring=fun_scaled_neg_MAPE, refit=False, verbose=True, n_jobs=-1)
tuning_details = fun_fit_tuning(grid_search, X_train, y_train, file_name=optimization_problem + "_NN")

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(grid_search, X_train, y_train)
model_results_dict.update(tuning_details)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(grid_search, param_grid)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


{'Parameter combinations': 72,
 'Total tuning time': '2h, 31m',
 'Grid search total fit time': '9h, 56m',
 'Grid search total prediction time': '3m, 31s'}

CV MAPE (scaled) train data:  3.3938 %




**Best model / parameter combination:**

{'mlpregressor__alpha': 0.0001,
 'mlpregressor__batch_size': 32,
 'mlpregressor__early_stopping': True,
 'mlpregressor__learning_rate_init': 0.0001,
 'mlpregressor__solver': 'adam'}


**Cross validation scores of different parameter combinations:**

,alpha,solver,batch_size,learning_rate_init,early_stopping,mean_test_score,converted_mean_fit_time
0,0.0001,adam,32,0.0001,True,-0.033938,"11m, 50s"
1,0.0001,adam,128,0.0010,True,-0.034091,"4m, 28s"
2,0.0001,adam,64,0.0001,True,-0.035033,"7m, 32s"
3,0.0001,adam,64,0.0010,True,-0.035234,"7m, 2s"
4,0.0001,adam,128,0.0001,True,-0.035436,"5m, 55s"
5,0.0010,adam,128,0.0001,True,-0.035453,"4m, 21s"
6,0.0010,adam,64,0.0001,True,-0.035907,"6m, 8s"
7,0.0010,adam,128,0.0010,True,-0.036665,"5m, 12s"
8,0.0001,adam,32,0.0010,True,-0.037069,"16m, 51s"
9,0.0010,adam,32,0.0001,True,-0.037208,"15m, 23s"


,alpha,solver,batch_size,learning_rate_init,early_stopping,mean_test_score,converted_mean_fit_time
60,0.0100,sgd,128,0.0010,False,-0.108731,"2m, 15s"
61,0.0010,sgd,128,0.0010,False,-0.109539,"2m, 52s"
62,0.0001,sgd,128,0.0010,False,-0.109619,"2m, 59s"
63,0.0100,sgd,32,0.0001,False,-0.124191,"4m, 51s"
64,0.0010,sgd,32,0.0001,False,-0.135576,"4m, 37s"
65,0.0001,sgd,32,0.0001,False,-0.135787,"5m, 1s"
66,0.0100,sgd,64,0.0001,False,-0.157994,"2m, 58s"
67,0.0010,sgd,64,0.0001,False,-0.160140,"3m, 13s"
68,0.0001,sgd,64,0.0001,False,-0.160226,"3m, 21s"
69,0.0100,sgd,128,0.0001,False,-0.187103,"2m, 9s"


**Test Score**

In [3]:
# Load best parameters of the model
best_params = fun_load_best_params(optimization_problem, model_abbreviation="NN")

# Create pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(hidden_layer_sizes=(256, 128, 64), activation="relu", learning_rate="adaptive",
                                  max_iter=1000, random_state=42))
pipe.set_params(**best_params)

# Estimate model performance with cross-validation on the train set and get scores on test set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(pipe, X_train, y_train, X_test, y_test, compute_test_scores=True)

{'mlpregressor__alpha': 0.0001,
 'mlpregressor__batch_size': 32,
 'mlpregressor__early_stopping': True,
 'mlpregressor__learning_rate_init': 0.0001,
 'mlpregressor__solver': 'adam'}

CV MAPE (scaled) train data:  3.1580999999999997 %
CV RMSE (scaled) train data: 0.0144
CV computation time: 41m, 28s

MAPE (scaled) test data:  3.3909000000000002 %
RMSE (scaled) test data: 0.0158
Model fit time: 1h, 9m
Model prediction time: 3s



**MAPE and RMSE on test data per instance size:**

Number Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,3.9578,3.8291,4.025,3.7349,3.5219,3.1038,3.3604,2.9044,2.8972,3.3909
RMSE,0.0204,0.0182,0.019,0.0168,0.0162,0.0140,0.0153,0.0129,0.0126,0.0158


In [83]:
# Create run times Data Frame
run_times_df = pd.DataFrame({key: [model_results_dict[key]] for key in ["CV computation time", "Model fit time", "Model prediction time"]})

# Create Data Frames with MAPE and RMSE scores of train and test set
mape_scores = model_results_dict["MAPE"]
mape_scores = {key: round(mape_scores[key], 2) for key in mape_scores.keys()} # Round MAPE scores
rmse_scores = model_results_dict["RMSE"]
scores_df = pd.DataFrame(data=[mape_scores.values(), rmse_scores.values()], columns=["Train score", "Test score"], index=["MAPE", "RMSE"])

# Get Data Frame with scores per instance size and display all Data Frames
cat_scores_df = model_results_dict["Scores per instance size"]
cat_scores_df.loc["MAPE"] = np.round(cat_scores_df.loc["MAPE"], 2) # Round MAPE scores
display(run_times_df, scores_df, cat_scores_df)

# Save data frames with results into an excel file
file_path = "04_test_results/" + optimization_problem + "_results.xlsx"

# Use ExcelWriter to write multiple DataFrames to the same file
with pd.ExcelWriter(file_path) as writer:
    run_times_df.to_excel(writer, sheet_name="run_times")
    scores_df.to_excel(writer, sheet_name="scores")
    cat_scores_df.to_excel(writer, sheet_name="cat_scores")

# Print total script run time
print("Total script computation time:", fun_convert_time(start=start_script, end=time.time() - 134 * 60))

,CV computation time,Model fit time,Model prediction time
0,"41m, 28s","1h, 9m",3s


,Train Score,Test Score
MAPE,3.1600,3.3900
RMSE,0.0144,0.0158


Number Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,3.9600,3.8300,4.030,3.7300,3.5200,3.100,3.3600,2.9000,2.9000,3.3900
RMSE,0.0204,0.0182,0.019,0.0168,0.0162,0.014,0.0153,0.0129,0.0126,0.0158


Total script computation time: 4h, 25m
